In [1]:
ENV["GKS_ENCODING"] = "utf-8"
using Plots
using LadderDGA
using Logging

io = stdout
metafmt(level::Logging.LogLevel, _module, group, id, file, line) = Logging.default_metafmt(level, nothing, group,     id, nothing, nothing)
logger = ConsoleLogger(io, Logging.Info, meta_formatter=metafmt, show_limited=true, right_justify=0)
global_logger(logger);

┌ Info: Precompiling LadderDGA [78e40beb-bf89-4c0e-9d2b-bee278912f2b]
└ @ Base loading.jl:1423


# Setup and read DMFT quantities

In [51]:
using TimerOutputs

using Pkg
Pkg.activate("/home/julian/Hamburg/Julia_lDGA/LadderDGA.jl")
using LadderDGA

cfg_file =  "/home/julian/Hamburg/Julia_lDGA/lDGA_shift_tests/data/40_40_s0_b14_u1.0/config_j.toml";
mP, sP, env, kGridsStr = readConfig(cfg_file);

impQ_sp, impQ_ch, gImp, kGridLoc, kG, gLoc, gLoc_fft, Σ_loc, FUpDo = setup_LDGA(kGridsStr[1], mP, sP, env);
@info "local"
bubbleLoc = calc_bubble(gImp, kGridLoc, mP, sP);
locQ_sp = calc_χ_trilex(impQ_sp.Γ, bubbleLoc, kGridLoc, mP.U, mP, sP);
locQ_ch = calc_χ_trilex(impQ_ch.Γ, bubbleLoc, kGridLoc, -mP.U, mP, sP);


Σ_ladderLoc = calc_Σ(locQ_sp, locQ_ch, bubbleLoc, gImp, FUpDo, kGridLoc, mP, sP)
Σ_ladderLoc = Σ_ladderLoc .+ mP.n * mP.U/2.0;

# ladder quantities
@info "bubble"
bubble = calc_bubble(gLoc_fft, kG, mP, sP);
@info "chi"
nlQ_sp = calc_χ_trilex(impQ_sp.Γ, bubble, kG, mP.U, mP, sP);
nlQ_ch = calc_χ_trilex(impQ_ch.Γ, bubble, kG, -mP.U, mP, sP);
@info "Σ"
Σ_ladder = calc_Σ(nlQ_sp, nlQ_ch, bubble, gLoc_fft, FUpDo, kG, mP, sP)
Σ_ladder = Σ_loc_correction(Σ_ladder,Σ_ladderLoc,Σ_loc);

[ Info: Reading Inputs...
[ Info: Setting up calculation for kGrid 3Dsc-0.2041241452319315 of size 4
[ Info: setting usable ranges of sp and ch channel from 28:54 and 28:54 to the same range of 28:54
[ Warning: TODO: update local omega sum with correction
[ Warning: using unoptimized sum_freq
[ Warning: using unoptimized sum_freq
┌ Info: Inputs Read. Starting Computation.
│ Local susceptibilities with ranges are:
│ χLoc_sp(28:54) = 0.2147, χLoc_ch(28:54) = 0.1365
└ sum χupup check (fit, tail sub, tail sub + fit, expected): 0.1755788445327572 ?≈? 0.21520636358989342 ?=? 0.21520636358989342 ?≈? 0.25"
[ Info: local
[ Info: bubble
[ Info: chi
[ Info: Σ


  Activating project at `~/Hamburg/Julia_lDGA/LadderDGA.jl`


In [52]:
println("Compare with energies_dmft_nusum.dat, column 3 and 5, last line")
iνₙ = LadderDGA.iν_array(mP.β, 0:(sP.n_iν-1))
iνₙ, GImp    = LadderDGA.readGImp(env.inputDir * "/gm_wim", only_positive=true)
ϵₖ, Vₖ, μ    = LadderDGA.read_anderson_parameters(env.inputDir * "/hubb.andpar");
E_kin_ED, E_pot_ED  = LadderDGA.calc_E_ED(iνₙ[1:20], ϵₖ, Vₖ, GImp[1:20], mP)
E_kin_ED_large, E_pot_ED_large  = LadderDGA.calc_E_ED(iνₙ[1:5000], ϵₖ, Vₖ, GImp[1:5000], mP)

Compare with energies_dmft_nusum.dat, column 3 and 5, last line


(-0.7546239818666916, 0.20374739512350937)

In [53]:
ep_loc, ek_loc = LadderDGA.calc_E(Σ_loc[:,1:sP.n_iν], kG, mP, sP);

LoadError: BoundsError: attempt to access 162-element Vector{ComplexF64} at index [1:162, 1:40]

In [54]:
iν_array = LadderDGA.iν_array
G_from_Σ = LadderDGA.G_from_Σ
flatten_2D = LadderDGA.flatten_2D

function calc_E(Σ, kG, mP::ModelParameters, sP::SimulationParameters)
    #println("TODO: E_pot function has to be tested")
    #println("TODO: use GNew/GLoc/GImp instead of Sigma")
    #println("TODO: make frequency summation with sum_freq optional")
    νmax = size(Σ,2)
    νGrid = 0:(νmax-1)
    iν_n = iν_array(mP.β, νGrid)
    Σ_hartree = mP.n * mP.U/2
    Σ_corr = Σ .+ Σ_hartree

    E_kin_tail_c = [zeros(size(kG.ϵkGrid)), (kG.ϵkGrid .+ Σ_hartree .- mP.μ)]
    E_pot_tail_c = [zeros(size(kG.ϵkGrid)),
                    (mP.U^2 * 0.5 * mP.n * (1-0.5*mP.n) .+ Σ_hartree .* (kG.ϵkGrid .+ Σ_hartree .- mP.μ))]
    tail = [1 ./ (iν_n .^ n) for n in 1:length(E_kin_tail_c)]
    E_pot_tail = sum(E_pot_tail_c[i] .* tail[i]' for i in 1:length(tail))
    E_kin_tail = sum(E_kin_tail_c[i] .* tail[i]' for i in 1:length(tail))
    E_pot_tail_inv = sum((mP.β/2)  .* [Σ_hartree .* ones(size(kG.ϵkGrid)), (-mP.β/2) .* E_pot_tail_c[2]])
    E_kin_tail_inv = sum(map(x->x .* (mP.β/2) .* kG.ϵkGrid , [1, -(mP.β) .* E_kin_tail_c[2]]))

    @warn "possible bug in flatten_2D with new axis, use calc_E_pot, calc_E_kin instead"
    G_corr = flatten_2D(G_from_Σ(Σ_corr, kG.ϵkGrid, νGrid, mP))';
    E_pot = real.(G_corr .* Σ_corr .- 0.0); #E_pot_tail
    E_kin = kG.ϵkGrid .* real.(G_corr .- 0.0); #E_kin_tail
#E_pot_tail_inv
#E_kin_tail_inv
    E_pot = [kintegrate(kG, 2 .* sum(E_pot[:,1:i], dims=[2])[:,1] .+ 0.0) for i in 1:νmax] ./ mP.β
    E_kin = [kintegrate(kG, 4 .* sum(E_kin[:,1:i], dims=[2])[:,1] .+ 0.0) for i in 1:νmax] ./ mP.β
    return E_kin, E_pot
end

calc_E (generic function with 1 method)

In [134]:
iν_array = LadderDGA.iν_array
G_from_Σ = LadderDGA.G_from_Σ
flatten_2D = LadderDGA.flatten_2D

Σ = Σ_ladder
νmax = size(Σ,2)
νGrid = 0:(νmax-1)
iν_n = iν_array(mP.β, νGrid)
Σ_hartree = mP.n * mP.U/2
Σ_corr = Σ .+ Σ_hartree

E_kin_tail_c = [zeros(size(kG.ϵkGrid)), (kG.ϵkGrid .+ Σ_hartree .- mP.μ)]
E_pot_tail_c = [zeros(size(kG.ϵkGrid)),
                (mP.U^2 * 0.5 * mP.n * (1-0.5*mP.n) .+ Σ_hartree .* (kG.ϵkGrid .+ Σ_hartree .- mP.μ))]
tail = [1 ./ (iν_n .^ n) for n in 1:length(E_kin_tail_c)]
E_pot_tail = sum(E_pot_tail_c[i] .* transpose(tail[i]) for i in 1:length(tail))
E_kin_tail = sum(E_kin_tail_c[i] .* transpose(tail[i]) for i in 1:length(tail))
E_pot_tail_inv = sum((mP.β/2)  .* [Σ_hartree .* ones(size(kG.ϵkGrid)), (-mP.β/2) .* E_pot_tail_c[2]])
E_kin_tail_inv = sum(map(x->x .* (mP.β/2) .* kG.ϵkGrid , [1, -(mP.β) .* E_kin_tail_c[2]]))

@warn "possible bug in flatten_2D with new axis, use calc_E_pot, calc_E_kin instead"
G_corr = transpose(flatten_2D(G_from_Σ(Σ_corr, kG.ϵkGrid, νGrid, mP)));
E_pot_full = real.(G_corr .* Σ_corr .- E_pot_tail);
E_kin_full = kG.ϵkGrid .* real.(G_corr .- E_kin_tail);
#E_pot_tail_inv
#E_kin_tail_inv
t = 0.5*kintegrate(kG, Σ_hartree .* ones(length(kG.kMult)))
E_pot = [kintegrate(kG, 2 .* sum(E_pot_full[:,1:i], dims=[2])[:,1] .+ E_pot_tail_inv) for i in 1:νmax] ./ mP.β
E_kin = [kintegrate(kG, 4 .* sum(E_kin_full[:,1:i], dims=[2])[:,1] .+ E_kin_tail_inv) for i in 1:νmax] ./ mP.β;

[ Warning: possible bug in flatten_2D with new axis, use calc_E_pot, calc_E_kin instead


In [135]:
E_kin

40-element Vector{Float64}:
 -0.40386947986245864
 -0.2892353809165824
 -0.25989552844586755
 -0.24925155300043317
 -0.2445641952409264
 -0.242213429138388
 -0.24091720351281704
 -0.2401486745466213
 -0.23966614725843596
 -0.23934881953609533
 -0.23913200654808128
 -0.23897905292547067
 -0.23886817812079905
  ⋮
 -0.23848518465319662
 -0.23848138858891232
 -0.23847806120695234
 -0.23847513190915048
 -0.23847254243651128
 -0.23847024445374543
 -0.23846819770384142
 -0.23846636864173837
 -0.2384647295680617
 -0.2384632584673579
 -0.23846194031458007
 -0.238460772693047

In [124]:
E_kin_tail[:,1]

10-element Vector{ComplexF64}:
       24.3221496057386 + 0.0im
      16.21476640382573 + 0.0im
      8.107383201912869 + 0.0im
 1.1023932874275725e-15 + 0.0im
      8.107383201912866 + 0.0im
 1.1023932874275725e-15 + 0.0im
     -8.107383201912864 - 0.0im
     -8.107383201912866 - 0.0im
     -16.21476640382573 - 0.0im
      -24.3221496057386 - 0.0im

In [132]:
E_kin_tail_c[2]

10-element Vector{Float64}:
 -1.224744871391589
 -0.816496580927726
 -0.40824829046386313
 -5.551115123125783e-17
 -0.408248290463863
 -5.551115123125783e-17
  0.4082482904638629
  0.408248290463863
  0.816496580927726
  1.224744871391589

In [131]:
E_pot_tail_c[2]

10-element Vector{Float64}:
 -0.36237243569579447
 -0.15824829046386302
  0.045875854768068436
  0.24999999999999997
  0.04587585476806849
  0.24999999999999997
  0.45412414523193145
  0.4541241452319315
  0.658248290463863
  0.8623724356957945

In [83]:
G_corr[:,1]

10-element Vector{ComplexF64}:
   0.5981358381302362 - 0.2628210052630989im
   0.5887170462258846 - 0.5653143426740675im
   0.7329398154167804 - 1.095667239180746im
 1.258885768325216e-7 - 1.280614227876605im
   0.7329398154167802 - 1.095667239180746im
 1.258885768325216e-7 - 1.280614227876605im
  -0.7329397404682071 - 1.095667394931361im
  -0.7329397404682074 - 1.095667394931361im
  -0.5887170402739587 - 0.565314452151864im
  -0.5981358505100658 - 0.2628210484007997im

In [92]:
Σ_corr[:,1] 

10-element Vector{ComplexF64}:
 0.32343775391944485 - 0.39133514858361995im
 0.43276006489382857 - 0.6242067114703185im
  0.4864579356323736 - 0.4061323417775565im
  0.4999999232373996 - 0.5564758096122363im
  0.4864579356323735 - 0.40613234177755675im
  0.4999999232373996 - 0.5564758096122363im
  0.5135419650597346 - 0.40613234743033755im
  0.5135419650597346 - 0.40613234743033744im
  0.5672397712639117 - 0.6242067270601005im
  0.6765621520230242 - 0.3913351955732436im

In [93]:
E_kin

40-element Vector{Float64}:
 -1.1131177653588231
 -1.0772890314680985
 -1.0763191104172383
 -1.0801495897778521
 -1.0842183836911399
 -1.0877291736670835
 -1.0906296834586513
 -1.0930133690946615
 -1.0949849881230724
 -1.0966323370918576
 -1.0980237968600712
 -1.0992115772365085
 -1.100235499688631
  ⋮
 -1.1073715808907119
 -1.1075715332566192
 -1.1077588127775067
 -1.107934580452619
 -1.1080998603966652
 -1.108255559358764
 -1.1084024830531987
 -1.1085413499414023
 -1.1086728030586614
 -1.1087974205420454
 -1.1089157259827065
 -1.1090282017333086

In [90]:
E_pot

40-element Vector{Float64}:
 0.010528697147643111
 0.04472763295057478
 0.04480609550804776
 0.040311877621191816
 0.035693511890049275
 0.03171730797006362
 0.02842117405418523
 0.025700387148712842
 0.023440373741583507
 0.021544945071441834
 0.019938736439657788
 0.018563825773061364
 0.01737575944840381
 ⋮
 0.00901649309747796
 0.008779103292173305
 0.008556326648568369
 0.008346782984566354
 0.008149252691348252
 0.00796266232696774
 0.007786084985654977
 0.007618768434481663
 0.007460230698234914
 0.007310523368650825
 0.0071709623912072406
 0.007046383640062969

40-element Vector{Float64}:
 -1.1131177653588231
 -1.0772890314680985
 -1.0763191104172383
 -1.0801495897778521
 -1.0842183836911399
 -1.0877291736670835
 -1.0906296834586513
 -1.0930133690946615
 -1.0949849881230724
 -1.0966323370918576
 -1.0980237968600712
 -1.0992115772365085
 -1.100235499688631
  ⋮
 -1.1073715808907119
 -1.1075715332566192
 -1.1077588127775067
 -1.107934580452619
 -1.1080998603966652
 -1.108255559358764
 -1.1084024830531987
 -1.1085413499414023
 -1.1086728030586614
 -1.1087974205420454
 -1.1089157259827065
 -1.1090282017333086

In [34]:
nlQ_ch.γ[:,40,41]

10-element Vector{ComplexF64}:
 0.7488039540800255 - 5.6199551715752806e-9im
 0.8181748064351114 - 3.589513281450801e-9im
 0.7582653432147911 - 6.369175004909619e-9im
 0.7931039823162078 - 6.08530122879078e-9im
  0.758265343214791 - 6.3691750103849e-9im
 0.7931039823162077 - 6.085301232067452e-9im
 0.7262271996435677 - 9.10207003172385e-9im
 0.7262271996435676 - 9.102070029276389e-9im
 0.7507683892807774 - 9.623841523436265e-9im
 0.6236329353752598 - 1.444101633047643e-8im

In [35]:
E_kin_full .* 2

LoadError: UndefVarError: E_kin_full not defined

In [36]:
E_kin, E_pot = calc_E(Σ_ladder, kG, mP, sP);

[ Warning: possible bug in flatten_2D with new axis, use calc_E_pot, calc_E_kin instead


In [37]:
E_kin

40-element Vector{Float64}:
 -0.07236920080087601
 -0.11535037255924395
 -0.14275456493572541
 -0.1610628950789639
 -0.17389052000352964
 -0.1832649859404447
 -0.1903639310771406
 -0.1959012169477909
 -0.2003281303549566
 -0.20394112192377714
 -0.20694167853929765
 -0.20947090494601403
 -0.2116302469202535
  ⋮
 -0.22629040807463932
 -0.22669430480896846
 -0.22707238478156813
 -0.2274270420703907
 -0.22776038478996274
 -0.22807427663865312
 -0.2283703715376558
 -0.2286501427556977
 -0.22891490772874332
 -0.22916584976730095
 -0.22940403835745146
 -0.22963045200116264

In [38]:
E_pot .* 4

40-element Vector{Float64}:
 0.27729726674885735
 0.44766074869990585
 0.5573017890193098
 0.6314892884123589
 0.6840910775143723
 0.7229154916130645
 0.7525466265869046
 0.7757983531607603
 0.7944713560165078
 0.8097615750372078
 0.8224896637319123
 0.8332350665748544
 0.8424172721092917
 ⋮
 0.9041804814795137
 0.9058055040479215
 0.9073119644003903
 0.9087090218318147
 0.9100042581083475
 0.9112036915476888
 0.9123116525093761
 0.9133304312040195
 0.9142594697301759
 0.9150935616979805
 0.9158185176867547
 0.9163992408028572

In [39]:
subtract_tail = LadderDGA.subtract_tail
iωn = 1im .* 2 .* (-sP.n_iω:sP.n_iω)[usable_ω] .* π ./ mP.β
plot(real(subtract_tail(impQ_sp.χ_ω[usable_ω], E_kin_ED_large,iωn)), markershape=:circle)
plot!(real(subtract_tail(impQ_ch.χ_ω[usable_ω], E_kin_ED_large, iωn )), markershape=:hexagon)
plot!(real(subtract_tail(0.5*(impQ_sp.χ_ω .+ impQ_ch.χ_ω )[usable_ω], E_kin_ED_large, iωn)))

LoadError: UndefVarError: usable_ω not defined